# Create KB and dataset

In [1]:
%cd ..

/home/codeholder/code/python-playground/app_noisemon


# 1. Prelude

In [92]:
import json
import os
from pathlib import Path
from tqdm import tqdm
from collections import defaultdict
import numpy as np

import spacy
from spacy.kb import KnowledgeBase
from spacy.tokens import Span, DocBin, Doc
from spacy.vocab import Vocab
from spacy.lang.ru import Russian
from typing import List, Callable, Iterable, Iterator, Optional, Dict, Union
from spacy.language import Language
from spacy.pipeline.trainable_pipe import TrainablePipe
from spacy.pipeline.pipe import deserialize_config
from spacy.tokens import Doc
from spacy.vocab import Vocab
from spacy.training import Example, validate_examples, validate_get_examples
from spacy import util, Errors
from spacy.util import minibatch
from thinc.api import Model, Config, set_dropout_rate, Optimizer





# from thinc.layers import TransformerListener

from wasabi import Printer
msg = Printer()

from scripts.convert_labelstudio_to_spacy import LabelStudioToSpacyConverter

In [217]:
512-2-128

382

In [246]:

DEFAULT_CONFIG_TRF = """
[transformer]
max_batch_items = 512

[transformer.model]
@architectures = "spacy-transformers.TransformerModel.v1"
name = "cointegrated/rubert-tiny"

[transformer.model.get_spans]
#@span_getters = "spacy-transformers.strided_spans.v1"
#window = 382
#stride = 128
@span_getters = "spacy-transformers.sent_spans.v1"

[transformer.model.tokenizer_config]
use_fast = true
return_tensors='pt'

"""

In [247]:
DEFAULT_CONFIG = Config().from_str(DEFAULT_CONFIG_TRF)

In [248]:
nlp = Russian()
config = {"punct_chars": ["\n\n", "\n"]}
nlp.add_pipe("sentencizer", config=config)
trf = nlp.add_pipe("transformer", config=DEFAULT_CONFIG["transformer"])
trf.model.initialize()

Initialized a transformer model
Init model method is called


Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


OOkay
PreTrainedTokenizerFast(name_or_path='cointegrated/rubert-tiny', vocab_size=29564, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [249]:
nlp.pipeline

[('sentencizer', <spacy.pipeline.sentencizer.Sentencizer at 0x7fee29dd1840>),
 ('transformer',
  <spacy_transformers.pipeline_component.Transformer at 0x7fee29dd9ae0>)]

In [250]:
doc = nlp("Какой-то текст 1,2,3")

In [251]:
type(doc._.trf_data.model_output.last_hidden_state)

numpy.ndarray

In [223]:
emb = doc._.trf_data.model_output.last_hidden_state
emb = emb / np.linalg.norm(emb)
emb = emb.squeeze()
emb.shape

(12, 312)

In [224]:
emb[[1,2,3]].shape

(3, 312)

In [225]:
emb = np.mean(emb, axis=0)
emb.shape

(312,)

In [226]:

nlp_model_vector_size = 312
kb = KnowledgeBase(vocab=nlp.vocab, entity_vector_length=nlp_model_vector_size)

output = []
converter = LabelStudioToSpacyConverter(nlp=nlp)
converter.ls_label_map = {
    "ORG": "ORG"
}

In [227]:
doc._.trf_data.wordpieces.strings[0]

['[CLS]', 'Како', '##й', '-', 'то', 'текст', '1', ',', '2', ',', '3', '[SEP]']

In [228]:
import spacy_alignments as tokenizations
print(doc._.trf_data.wordpieces.strings[0])
print([str(token) for token in doc])
a2b, b2a = tokenizations.get_alignments([str(token) for token in doc], doc._.trf_data.wordpieces.strings[0])
print(a2b)
print(b2a)

['[CLS]', 'Како', '##й', '-', 'то', 'текст', '1', ',', '2', ',', '3', '[SEP]']
['Какой', '-', 'то', 'текст', '1,2,3']
[[1, 2], [3], [4], [5], [6, 7, 8, 9, 10]]
[[], [0], [0], [1], [2], [3], [4], [4], [4], [4], [4], []]


In [229]:
print(doc[0])


Какой


# 2. Data import

In [264]:
input_path = Path("./data/05-labeled")
output_folder = Path("./corpus/")
kb_path = Path("./corpus/noisemon_kb")

In [265]:
files = list(input_path.glob("*.json"))
if files:
    input_path = max(files, key=os.path.getctime)
else:
    msg.fail(f"Directory {input_path} is empty")

In [266]:
data = json.loads(input_path.read_text())

In [267]:
data[0]

{'id': 334577,
 'annotations': [{'id': 27,
   'completed_by': {'id': 1,
    'email': 'maksim_ermakov@protonmail.com',
    'first_name': '',
    'last_name': ''},
   'result': [{'value': {'start': 36,
      'end': 41,
      'text': 'Русал',
      'labels': ['ORG']},
     'id': '91483591258789',
     'from_name': 'ner',
     'to_name': 'text',
     'type': 'labels'},
    {'value': {'start': 36, 'end': 41, 'text': ['Q1642605']},
     'id': '91483591258789',
     'from_name': 'entity',
     'to_name': 'text',
     'type': 'textarea'},
    {'value': {'start': 522,
      'end': 553,
      'text': 'Лондонской бирже металлов (LME)',
      'labels': ['ORG']},
     'id': '26347872065953',
     'from_name': 'ner',
     'to_name': 'text',
     'type': 'labels'},
    {'value': {'start': 522, 'end': 553, 'text': ['Q952937']},
     'id': '26347872065953',
     'from_name': 'entity',
     'to_name': 'text',
     'type': 'textarea'}],
   'was_cancelled': False,
   'ground_truth': False,
   'created_at'

# 3. Processing

In [268]:
np.mean([np.mean([[1,2,3], [1,2,4]], axis=0).tolist(), [2,2,3]], axis=0)

array([1.5 , 2.  , 3.25])

In [269]:
np.mean([[1,2,3], [1,2,4], [2,2,3]], axis=0)

array([1.33333333, 2.        , 3.33333333])

In [270]:
id_to_qid_name_pair

defaultdict(dict,
            {'74520158391160': {'text': 'Открытие Капитал',
              'qid': 'Q108398998'},
             '18591563568703': {'text': 'Роснефти', 'qid': 'Q1141123'},
             '27292557371956': {'text': 'Роснефти', 'qid': 'Q1141123'}})

In [271]:
qid_to_vector_list = defaultdict(list)

In [272]:
qid_to_alias = defaultdict(set)

In [273]:
output = []
for labelstudio in tqdm(data[::-1]):
    id_to_qid_name_pair = defaultdict(dict)
    doc = converter.create_spacy_doc(labelstudio)
    
    emb = doc._.trf_data.model_output.last_hidden_state
    emb = emb / np.linalg.norm(emb)
    emb = emb.squeeze()
    
    entities = []

    # 1. Matching named entities with QIDs
    for chunk in labelstudio["annotations"][0]["result"]:
        if chunk["from_name"] == "ner":
            id_to_qid_name_pair[chunk["id"]]["text"] = chunk["value"]["text"]
        if chunk["from_name"] == "entity":
            id_to_qid_name_pair[chunk["id"]]["qid"] = chunk["value"]["text"][0]
            
    # 2. Create ents and assign kb_id to ents
    doc_alignment, _ = tokenizations.get_alignments([str(token) for token in doc], doc._.trf_data.wordpieces.strings[0])
    for chunk in labelstudio["annotations"][0]["result"]:
        if chunk["from_name"] == "ner":
            # 1. Try to create an entity
            QID = id_to_qid_name_pair[chunk["id"]].get("qid", None)
            if not QID:
                print(f"{id_to_qid_name_pair[chunk['id']]['text']} has no matching QID")
            try:
                entity = doc.char_span(
                    chunk["value"]["start"], 
                    chunk["value"]["end"], 
                    label=chunk["value"]["labels"][0],
                    kb_id=QID
                )
                assert entity != None, "Entity failed to be created. Probably misaligned markup"
                
                entities.append(entity)
            except:
                msg.fail("REsult:", chunk)
                msg.fail("Doc:", doc)
                msg.fail("----------")
                continue
            # 2. Get entity vector
            ent_idxs = [idx for list_of_indices in doc_alignment[entity.start: entity.end] for idx in list_of_indices]
#             print(entity.start, entity.end, ent_idxs, doc_alignment)
            entity_vector = np.mean(emb[ent_idxs], axis=0)
            qid_to_vector_list[QID].append(entity_vector)
            qid_to_alias[QID].add(entity.text)

    doc.ents = entities
    output.append(doc)

100%|████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 85.78it/s]


In [274]:
doc

RUAL отчет
Чистая прибыль компании «Русал» за январь—сентябрь 2017 года по ​по международным стандартам финансовой отчетности (МСФО) ​составила ​$782 млн, превысив на ​46,4% аналогичный показатель прошлого года. Об этом говорится в отчете о финансовых результатах, опубликованном на сайте компании.
При этом выручка компании за третий квартал увеличилась на 19,4% и составила $2,46 млрд, а за девять месяцев текущего года — на 21,3%, до $7,224 млрд.
Росту финансовых показателей способствовало увеличение цены алюминия на Лондонской бирже металлов (LME), говорится в отчете.

In [275]:
entity.start

94

In [257]:
doc_alignment[entity.start:entity.end]

[[83, 84, 85, 86]]

In [258]:
emb.shape

(140, 312)

In [259]:
doc._.trf_data.wordpieces.strings[1]

IndexError: list index out of range

In [276]:
# populating KB with vectors
for QID, vectors in qid_to_vector_list.items():
    kb.add_entity(entity=QID, entity_vector=np.mean(vectors, axis=0), freq=265)

In [277]:
# Populating aliases   
for QID, aliases in qid_to_alias.items():
    for alias in aliases:
        kb.add_alias(alias=alias, entities=[QID], probabilities=[1])

### At this point we have populated knowledge base `kb`, list of docs with ner and nel markup `output`

In [278]:
kb.get_alias_candidates("CME Group")[0].entity_

'Q1023876'

In [279]:
output[57].ents[0].kb_id_

'Q205012'

In [280]:
print(f"Entities in the KB: {kb.get_entity_strings()}")
print(f"Aliases in the KB: {kb.get_alias_strings()}")

Entities in the KB: ['Q4218402', 'Q660770', 'Q1141123', 'Q30893504', 'Q193199', 'Q58707', 'Q4161561', 'Q3063197', 'Q4044421', 'Q2369311', 'Q487907', 'Q2632892', 'Q1915579', 'Q1720713', 'Q223799', 'Q1071853', 'Q2380266', 'Q1477012', 'Q841458', 'Q108398998', 'Q1840188', 'Q251546', 'Q1023876', 'Q4047736', 'Q4304175', 'Q108397344', 'Q768773', 'Q171240', 'Q182477', 'Q940518', 'Q2624680', 'Q102673', 'Q173395', 'Q483551', 'Q329347', 'Q294508', 'Q379271', 'Q1368919', 'Q1809133', 'Q1284261', 'Q4059809', 'Q4258608', 'Q1549389', 'Q1461799', 'Q108397243', 'Q1616858', 'Q2304119', 'Q638448', 'Q7907607', 'Q2035424', 'Q2005769', 'Q4038038', 'Q4244736', 'Q1355823', 'Q3656098', 'Q1642605', 'Q108352452', 'Q4513187', 'Q1967957', 'Q4389244', 'Q871308', 'Q1963801', 'Q952937', 'Q130879', 'Q108396966', 'Q727452', 'Q108398486', 'Q2309', 'Q4499024', 'Q4327204', 'Q131723', 'Q1884500', 'Q2116312', 'Q205012', 'Q567050', 'Q4102033', 'Q1781702', 'Q386414', 'Q4400200']
Aliases in the KB: ['РусГидро', 'UBS', 'ПАО "Тру

In [281]:
kb.to_disk(kb_path)

# nlp.to_disk(nlp_dir)

In [282]:
train_docs = DocBin()
dev_docs = DocBin()
test_docs = DocBin()


In [283]:
for i, doc in enumerate(output):
    if i % 10 in (1,3,5):
        dev_docs.add(doc)
    if i % 10 in (4,):
        test_docs.add(doc)
    else:
        train_docs.add(doc)

In [284]:
train_docs.to_disk(output_folder / "train.spacy")
test_docs.to_disk(output_folder / "test.spacy")
dev_docs.to_disk(output_folder / "dev.spacy")